In [1]:
import pandas as pd
import hashlib
import sqlite3
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
def get_endpoint() -> str:
    """
    Fetches the endpoint used in this utility, returns the JSON type
    Returns:
        url (str): the endpoint url
    """
    sparql = SPARQLWrapper("https://staging.gss-data.org.uk/sparql")
    sparql.setReturnFormat(JSON)
    return sparql

def get_sparql(query_text: str) -> pd.DataFrame:
    """
    Queries the set SPARQL endpoint the given query_text or returns a sqlite3
    cached version. It creates a new sqlite3 database per SPARQL endpoint, and
    there is no cache expiry. If you want to fetch fresh data, delete the
    database. Soz.
    Parameters:
        query_text (str): a SPARQL query
    Returns:
        results (DataFrame): the results in a pandas DataFrame.
    """
    # Really bad caching, but I digress
    query_hex = hashlib.sha224(query_text.encode()).hexdigest()
    endpoint_hex = hashlib.sha224(get_endpoint().endpoint.encode()).hexdigest()
    # find the database connection
    con = sqlite3.connect(f"{endpoint_hex}.db")
    # check if the query_hex exists
    exist_query = f"SELECT count(*) FROM sqlite_master WHERE type='table' AND name='{query_hex}';"
    exist_result = bool(con.execute(exist_query).fetchone()[0])  # 0 = False
    if exist_result:
        # results exist so fetch results
        df = pd.read_sql(f"SELECT * FROM [{query_hex}]", con=con)
    else:
        # results don't exist in db so get them from SPARQL
        sparql = get_endpoint()
        sparql.setQuery(query_text)
        results = sparql.queryAndConvert()
        # normalise the json results
        df = pd.json_normalize(results['results']['bindings'])
        # store the results in db
        df.to_sql(query_hex, con)
        # log the query
        record = {
            "hash": [query_hex],
            "query": [query_text],
            "timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
        }
        pd.DataFrame.from_dict(record, orient="columns").to_sql(
            "manifest", con, if_exists="append", index=False
        )
    return df

def get_datasets() -> dict:
    """
    Function to get the datasets (i.e. qb:cubes) for the SPARQL endpoint.
    Returns:
        results (DataFrame): The list of qb:Cubes on a SPARQL endpoint.
    """
    query = """PREFIX pmdcat: <http://publishmydata.com/pmdcat#>
PREFIX dcterms: <http://purl.org/dc/terms/>
# Select details regarding all datasets, assume that if there's a catalog entry
# there is an associated dataset. Might be good to check if the GRAPH exists.
SELECT ?graphUrl ?catUrl ?datasetUrl ?catTitle ?catDesc 
WHERE {
	?catUrl pmdcat:graph ?graphUrl ;
              dcterms:title ?catTitle ;
              dcterms:description ?catDesc ;
              pmdcat:datasetContents ?datasetUrl .
}"""
    return get_sparql(query)

def get_components(graph_uri: str):
    """
    Function to get the list qb:componentProperty for a given graph containing a qb:cube.
    Arguments:
        graph_uri (str): string URI of for the graph containing the qb:cube
    Returns:
        results (DataFrame): The components of a qb:Cube
    """
    query = f"""PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# For a given cube's graph, select its components
SELECT ?graphUrl ?component ?kind ?definition
WHERE {{
    BIND(<{graph_uri}> as ?graphUrl) .
    GRAPH  ?graphUrl {{
        ?component qb:componentProperty ?definition .
        ?definition rdf:type ?kind .
    }}
}}"""
    return get_sparql(query)
# Compare dimensions
# Want to use urllib's object model to get the fragments of a url to find out
# things like the componentProperty type
# urllib.parse.urlparse(json_normalize(components).loc[3, 'kind.value'])
# > ParseResult(scheme='http', netloc='purl.org', path='/linked-data/cube', params='', query='', fragment='AttributeProperty')
# Also want to use urllib to extract the path, to get the dimension name



In [2]:
cubes = get_datasets()

In [3]:
cubes

,graphUrl.type,graphUrl.value,datasetUrl.type,datasetUrl.value,catUrl.type,catUrl.value,catDesc.xml:lang,catDesc.type,catDesc.value,catTitle.xml:lang,catTitle.type,catTitle.value
0,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,en,literal,A statistical report showing the impact of Cov...,en,literal,Statistics relating to Covid-19 and the immigr...
1,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,en,literal,Deaths involving coronavirus (COVID-19) in Sco...,en,literal,Deaths involving COVID-19 in Scotland
2,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,en,literal,Notifications to Care Inspectorate Wales of de...,en,literal,Notifications of deaths of residents related t...
3,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,en,literal,ONS Number of deaths in care homes notified to...,en,literal,Number of deaths in care homes notified to the...
4,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,en,literal,Deaths involving COVID-19 in the care sector f...,en,literal,Deaths involving COVID-19 in the care sector f...
...,...,...,...,...,...,...,...,...,...,...,...,...
207,uri,http://gss-data.org.uk/graph/gss_data/homeless...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,en,literal,The Northern Ireland Homelessness bulletin is ...,en,literal,NIHE - Homelessness Acceptances
208,uri,http://gss-data.org.uk/graph/gss_data/homeless...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,en,literal,The Northern Ireland Homelessness bulletin is ...,en,literal,NIHE - Homelessness Presentations
209,uri,http://gss-data.org.uk/graph/gss_data/homeless...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,uri,http://gss-data.org.uk/data/gss_data/homelessn...,en,literal,The Northern Ireland Homelessness bulletin is ...,en,literal,NIHE - Temporary Accommodation
210,uri,http://gss-data.org.uk/graph/gss_data/energy/b...,uri,http://gss-data.org.uk/data/gss_data/energy/be...,uri,http://gss-data.org.uk/data/gss_data/energy/be...,en,literal,Application statistics for the Renewable Heat ...,en,literal,"RHI deployment data - Application Numbers, dom..."


In [6]:
cubes.loc[19, "graphUrl.value"]

'http://gss-data.org.uk/graph/gss_data/covid-19/mmo-ad-hoc-statistical-release-uk-sea-fisheries-statistics'

In [7]:
get_components(cubes.loc[19, "graphUrl.value"])

,graphUrl.type,graphUrl.value,component.type,component.value,kind.type,kind.value,definition.type,definition.value
0,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/cube#measureType
1,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/sdmx/2009/dimensio...
2,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#AttributeProp...,uri,http://purl.org/linked-data/sdmx/2009/attribut...
3,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/sdmx/2009/dimensio...
4,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#AttributeProp...,uri,http://purl.org/linked-data/sdmx/2009/attribut...
5,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...
6,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...
7,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...
8,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#MeasureProperty,uri,http://gss-data.org.uk/def/measure/weight


In [8]:
components = get_components(cubes.loc[19, "graphUrl.value"])

In [10]:
url = components.loc[3, "kind.value"]

In [11]:
url

'http://purl.org/linked-data/cube#DimensionProperty'

In [15]:
from urllib.parse import urlparse


In [17]:
urlparse(url)

ParseResult(scheme='http', netloc='purl.org', path='/linked-data/cube', params='', query='', fragment='DimensionProperty')

In [19]:
urlparse(url).fragment

'DimensionProperty'

In [20]:
components['kind.value'].apply(lambda x: urlparse(x).fragment)

0    DimensionProperty
1    DimensionProperty
2    AttributeProperty
3    DimensionProperty
4    AttributeProperty
5    DimensionProperty
6    DimensionProperty
7    DimensionProperty
8      MeasureProperty
Name: kind.value, dtype: object

In [21]:
components['definition.value'].apply(lambda x: urlparse(x).fragment)

0                measureType
1                  refPeriod
2                unitMeasure
3                    refArea
4                  obsStatus
5    dimension/vessel-length
6    dimension/species-group
7       dimension/admin-port
8                           
Name: definition.value, dtype: object

In [23]:
components['definition.value']

0         http://purl.org/linked-data/cube#measureType
1    http://purl.org/linked-data/sdmx/2009/dimensio...
2    http://purl.org/linked-data/sdmx/2009/attribut...
3    http://purl.org/linked-data/sdmx/2009/dimensio...
4    http://purl.org/linked-data/sdmx/2009/attribut...
5    http://gss-data.org.uk/data/gss_data/covid-19/...
6    http://gss-data.org.uk/data/gss_data/covid-19/...
7    http://gss-data.org.uk/data/gss_data/covid-19/...
8            http://gss-data.org.uk/def/measure/weight
Name: definition.value, dtype: object

In [24]:
components.loc[6, 'definition.value']

'http://gss-data.org.uk/data/gss_data/covid-19/mmo-ad-hoc-statistical-release-uk-sea-fisheries-statistics#dimension/species-group'

In [25]:
components['definition.value'].apply(lambda x: urlparse(x).fragment)

0                measureType
1                  refPeriod
2                unitMeasure
3                    refArea
4                  obsStatus
5    dimension/vessel-length
6    dimension/species-group
7       dimension/admin-port
8                           
Name: definition.value, dtype: object

In [26]:
components['kind.value'].apply(lambda x: urlparse(x).fragment)

0    DimensionProperty
1    DimensionProperty
2    AttributeProperty
3    DimensionProperty
4    AttributeProperty
5    DimensionProperty
6    DimensionProperty
7    DimensionProperty
8      MeasureProperty
Name: kind.value, dtype: object

In [27]:
components.loc[components['kind.value'].apply(lambda x: urlparse(x).fragment) == 'DimensionProperty']

,index,graphUrl.type,graphUrl.value,component.type,component.value,kind.type,kind.value,definition.type,definition.value
0,0,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/cube#measureType
1,1,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/sdmx/2009/dimensio...
3,3,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://purl.org/linked-data/sdmx/2009/dimensio...
5,5,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...
6,6,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...
7,7,uri,http://gss-data.org.uk/graph/gss_data/covid-19...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...,uri,http://purl.org/linked-data/cube#DimensionProp...,uri,http://gss-data.org.uk/data/gss_data/covid-19/...


In [28]:
urlparse(components.loc[8, 'definition.value']).fragment

''

In [29]:
type(urlparse(components.loc[8, 'definition.value']).fragment)

str

In [31]:
len(urlparse(components.loc[8, 'definition.value']).fragment)

0

In [32]:
from urllib.parse import urlparse
def fetch_fragment(url: str) -> str:
    """ Function to parse a url when you can't depend on the fragment
    being the component of the property you want.
    Parameters:
        url (str): the url you want to parse
    Returns:
        fragment (str): the final part of a parsed url
    """
    parsed = urlparse(url)
    if len(parsed.fragment) == 0:
        return parsed.path.split("/")[-1]
    elif "/" in parsed.fragment:
        return parsed.fragment.split("/")[-1]
    else:
        return parsed.fragment



In [33]:
fetch_fragment(components.loc[8, 'definition.value'])

'weight'

In [34]:
components['definition.value'].apply(lambda x: fetch_fragment(x))

0      measureType
1        refPeriod
2      unitMeasure
3          refArea
4        obsStatus
5    vessel-length
6    species-group
7       admin-port
8           weight
Name: definition.value, dtype: object

In [35]:
components['kind.value'].apply(lambda x: fetch_fragment(x))

0    DimensionProperty
1    DimensionProperty
2    AttributeProperty
3    DimensionProperty
4    AttributeProperty
5    DimensionProperty
6    DimensionProperty
7    DimensionProperty
8      MeasureProperty
Name: kind.value, dtype: object

In [36]:
def get_fragment(url: str) -> str:
    """Function to parse a url when you can't depend on the fragment
    being the component of the property you want.
    Parameters:
        url (str): the url you want to parse
    Returns:
        fragment (str): the final part of a parsed url
    """
    parsed = urlparse(url)
    if len(parsed.fragment) == 0:
        return parsed.path.split("/")[-1]
    elif "/" in parsed.fragment:
        return parsed.fragment.split("/")[-1]
    else:
        return parsed.fragment



In [37]:
components.loc[
        components["kind.value"].apply(lambda x: get_fragment(x))
        == "DimensionProperty",
        "definition.value",
    ]

0         http://purl.org/linked-data/cube#measureType
1    http://purl.org/linked-data/sdmx/2009/dimensio...
3    http://purl.org/linked-data/sdmx/2009/dimensio...
5    http://gss-data.org.uk/data/gss_data/covid-19/...
6    http://gss-data.org.uk/data/gss_data/covid-19/...
7    http://gss-data.org.uk/data/gss_data/covid-19/...
Name: definition.value, dtype: object

In [38]:
type(components.loc[
        components["kind.value"].apply(lambda x: get_fragment(x))
        == "DimensionProperty",
        "definition.value",
    ])

pandas.core.series.Series

In [39]:
components.loc[
        components["kind.value"].apply(lambda x: get_fragment(x))
        == "DimensionProperty",
        "definition.value",
    ].apply(lambda x: get_fragment(x))

0      measureType
1        refPeriod
3          refArea
5    vessel-length
6    species-group
7       admin-port
Name: definition.value, dtype: object

In [40]:
list(components.loc[
        components["kind.value"].apply(lambda x: get_fragment(x))
        == "DimensionProperty",
        "definition.value",
    ].apply(lambda x: get_fragment(x)))

['measureType',
 'refPeriod',
 'refArea',
 'vessel-length',
 'species-group',
 'admin-port']